In [ ]:
!pip3 install -U pysr==0.18.2 pytorch_lightning

In [ ]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
#from pysr import PySRRegressor
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

In [ ]:
###### np.random.seed(0)
N = 100000
Nt = 10
X = 6 * np.random.rand(N, Nt, 5) - 3
y_i = X[..., 0] ** 2 + 6 * np.cos(2 * X[..., 2])
y = np.sum(y_i, axis=1) / y_i.shape[1]
z = y**2
X.shape, y.shape

In [ ]:
hidden = 100

max_epochs = 20
Xt = torch.tensor(X).float()
zt = torch.tensor(z).float()
hidden = 128
total_steps = 50000


def mlp(size_in, size_out, act=nn.ReLU):
    return nn.Sequential(
        nn.Linear(size_in, hidden),
        act(),
        nn.Linear(hidden, hidden),
        act(),
        nn.Linear(hidden, hidden),
        act(),
        nn.Linear(hidden, size_out),
    )

class SumNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        X_train, X_test, z_train, z_test = train_test_split(Xt, zt, random_state=0)
        self.train_set = TensorDataset(X_train, z_train)
        self.test_set = TensorDataset(X_test, z_test)

        # The same inductive bias as above!
        self.g = mlp(5, 1)
        self.f = mlp(1, 1)

    def forward(self, x):
        y_i = self.g(x)[:, :, 0]
        y = torch.sum(y_i, dim=1, keepdim=True) / y_i.shape[1]
        z = self.f(y)
        return z[:, 0]

    ########################################################

    # PyTorch Lightning bookkeeping:
    def training_step(self, batch, batch_idx):
        x, z = batch
        predicted_z = self(x)
        loss = F.mse_loss(predicted_z, z)
        return loss

    def validation_step(self, batch, batch_idx):
        return self.training_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        scheduler = {'scheduler': torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2,
                                                                      steps_per_epoch=len(self.train_dataloader()),
                                                                      epochs=max_epochs,
                                                                      final_div_factor=1e4),
                     'interval': 'step'}
        return [optimizer], [scheduler]

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=128, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=256, num_workers=4)


pl.seed_everything(0)
model = SumNet()
trainer = pl.Trainer(max_epochs=max_epochs)
trainer.fit(model)

In [ ]:
from google.colab import files, drive
trainer.save_checkpoint("example.ckpt")
files.download("example.ckpt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Save and upload model

model = MyLightningModule(hparams)

trainer.fit(model)

trainer.save_checkpoint("example.ckpt")

new_model = MyModel.load_from_checkpoint(checkpoint_path="example.ckpt")